# Transformer

---
## 目的
Transformerネットワークを作成して，その構造と演算の中身について理解する．

## 対応するチャプター

## モジュールのインポート
プログラムの実行に必要なモジュールをインポートします．

In [ ]:
import numpy as np
import random
import math
import copy

## 演習タスクとデータセットの作成

今回は文字列としての足し算を行う計算機をTransformerで作成します．

まずは，足し算のデータを作成するデータセットクラスを作成します．
データは0から9までの数字と加算記号，開始，終了のフラグです．また，３桁の数字の足し算を行うため，各桁の値を１つずつランダムに生成して連結しています．

In [ ]:
word2id = {str(i): i for i in range(10)}
word2id.update({"<pad>": 10, "+": 11, "<eos>": 12})
id2word = {v: k for k, v in word2id.items()}

class CalcDataset:

    def transform(self, string, seq_len=7):
        tmp = []
        for i, c in enumerate(string):
            try:
                tmp.append(word2id[c])
            except:
                tmp += [word2id["<pad>"]] * (seq_len - i)
                break
        return tmp

    def __init__(self, data_num, train=True):
        self.data_num = data_num
        self.train = train
        self.data = []
        self.label = []

        for _ in range(data_num):
            x = int("".join([random.choice(list("0123456789")) for _ in range(random.randint(1, 3))] ))
            y = int("".join([random.choice(list("0123456789")) for _ in range(random.randint(1, 3))] ))
            left = ("{:*<7s}".format(str(x) + "+" + str(y))).replace("*", "<pad>")
            self.data.append(self.transform(left))

            z = x + y
            right = ("{:*<6s}".format(str(z))).replace("*", "<pad>")
            right = self.transform(right, seq_len=5)
            right = [12] + right
            right[right.index(10)] = 12
            self.label.append(right)
        
        self.data = np.asarray(self.data)
        self.label = np.asarray(self.label)

    def __getitem__(self, item):
        d = self.data[item]
        l = self.label[item]
        return d, l

    def __len__(self):
        return self.data.shape[0]

## Maskの作成

データに対するマスクを作成する関数を定義します．

デコーダは未来の情報を伝播しないようにMaskをかけます．

下図に示すように，Maskは同じ情報から作成されます．黒丸はマスクされた領域を表します．
例えば，「好き」という情報を入力した場合に，残りの「な/動物/は」を参照できません．
これは推論時未来の情報が与えられないためです．
そのため，Queryでは，入力の時刻より先のMemoryの情報に対してMaskをすることで，未来の情報を伝播させないようにします．
このマスクはデコーダのMasked Multi-Head Attentionで使われます．


<img src="https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/143078/4dab846e-19ac-3ed4-e16b-943b5921a47e.jpeg" width=40%>

In [ ]:
def encoder_mask(batch_size, src, size):
    _mask = src == word2id["<pad>"]
    mask = copy.deepcopy(_mask)
    mask[_mask == 1] = 0.0
    mask[_mask == 0] = 1.0
    return mask.reshape([mask.shape[0], 1, mask.shape[1]])

def decoder_mask(batch_size, size):
    _mask = np.triu(np.ones((size, size)), k=1)
    mask = copy.deepcopy(_mask)
    mask[_mask == 1] = 0.0
    mask[_mask == 0] = 1.0

    mask = mask.reshape([1, *mask.shape])
    mask = np.repeat(mask, batch_size, axis=0)

    return mask

def create_masks(batch_size, src, trg):
    src_mask = encoder_mask(batch_size, src, src.shape[1])

    if trg is not None:
        size = trg.shape[1]
        np_mask = decoder_mask(batch_size, size)
        trg_mask = np_mask
    else:
        trg_mask = None

    return src_mask, trg_mask

## Transformerの作成

### 概要
2017年に発表されたTransformerは，CNNやRNNなどを用いずAttention機構のみを用いたモデルです．
翻訳や文章生成などのタスクでRNNとSeq2seqモデルが主流でしたが，これらのモデルは逐次的に単語を処理するため学習時に並列計算できないという問題がありました．
また，長文に対してAttentionが使われていましたが，このAttentionはほとんどRNNと一緒に使われていました．
一方で，TransformerはAttention機構だけ使うことで，入出力の文章同士の広範囲な依存関係を捉える構造になっています．

モデルはSeq2seqと同様にエンコーダ・デコーダモデルです．
エンコーダでは，Multi-Head AttentionとFeed Forwardのブロックを$N$回スタックする構造です．
デコーダでは，それに加えMasked Multi-Head Attentionのブロックで構成されています．
Masked Multi-Head Attentionの学習時，デコーダは自己回帰を使用せず，全ターゲットを同時に入力し，全ターゲットを同時に予測します．
この時，予測すべきターゲットの情報が予測前のデコーダにリークしないようにMaskします．
評価時は自己回帰でターゲットを生成します．

<img src="https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/143078/badbcfc3-93c3-eb02-ae96-8b6256397c2a.png" width=35%>

### 活性化関数・全結合層の定義

まず，ネットワークの定義に必要となる関数や基本的な全結合層を定義します．
これらの定義や詳細については，前回までの演習を参照してください．

In [ ]:
def relu(x):
    return np.maximum(0, x)

def softmax(x, dim=0):
    x_max = np.max(x, axis=dim, keepdims=True)
    e_x = np.exp(x - x_max)
    x_sum = np.sum(e_x, axis=dim, keepdims=True)
    f_x = e_x / x_sum 
    return f_x

class Linear:
    def __init__(self, in_features, out_features, bias=True):
        self.in_features = in_features
        self.out_features = out_features
        self.use_bias = bias

        self.weight = np.random.randn(out_features, in_features)
        if self.use_bias:
            self.bias = np.random.randn(out_features)
        else:
            self.bias = None
    
    def __call__(self, x):
        if self.use_bias:
            return np.dot(x, self.weight.T) + self.bias
        else:
            return np.dot(x, self.weight.T)
        
class LinearEmbed:
    def __init__(self, in_features, out_features):
        self.in_features = in_features
        self.out_features = out_features
        
        self.weight = np.random.randn(in_features, out_features)
    
    def __call__(self, x):
        return np.dot(x, self.weight)

### Word EmbeddingとPositional Encoding

次に，Transformerへデータが入力された際に用いる，EmbeddingとPositional Encodingを定義します．

Embeddingは，各文字に対応するid列が入力され，それらに対してユニークな特徴をエンコードする層です．

Positonal Encoderは，Embedingした単語の埋め込み表現に対して，その1に対応した値を足し合わせることで位置情報を与える役割を持つ機構です．

In [ ]:
class Embedding:
    def __init__(self, num_embeddings, embedding_dim):
        self.num_embeddings = num_embeddings
        self.embedding_dim = embedding_dim

        self.embed = LinearEmbed(self.num_embeddings, self.embedding_dim)

    def __call__(self, x):
        h = self._onehot(x)
        h = self.embed(h)
        h[idx == 10, :] = 0 
        return h

    def _onehot(self, idx):
        return np.eye(self.num_embeddings)[idx]


class PositionalEncoder:
    def __init__(self, embedding_dim, max_seq_len=200):
        self.embedding_dim = embedding_dim
        pe = np.zeros((max_seq_len, embedding_dim))
        for pos in range(max_seq_len):
            for i in range(0, self.embedding_dim, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i) / self.embedding_dim)))
                pe[pos, i+1] = math.cos(pos / (10000 ** ((2 * (i + 1)) / self.embedding_dim)))
        self.pe = np.expand_dims(pe, axis=0)

    def __call__(self, x):
        h = x * math.sqrt(self.embedding_dim)
        seq_len = h.shape[1]
        pe = self.pe[:, :seq_len]
        h = h + pe
        return x

### Multi-Head Attention

#### 1. Scaled Dot-product Attention

Scaled Dot-Product Attentionは，Self-Attentionとも呼ばれ，Transformerの重要な機構です．

数式は以下の通りです．
$$ {\rm Attention}(Q, K, V)={\rm softmax} ( \frac{QK^{T}}{\sqrt{d_k} } ) V $$

ここで，$Q, K, V$はそれぞれQuery，Key，Valueです．また$d_k$はQueryの次元数を表します．この平方根$d_k$は，見てわかるように$Q, K$の特徴量をスケールする役割を持ちます．これは層数，すなわちスタックするブロック数(前述のN)が大きくなると，内積が大きくなり，softmax関数の勾配を計算すると非常に小さい値しか返さないためです．

図のように，QueryとKeyが行列乗算で計算された後，dの平方根でスケーリングした後，後述するMaskをかけます．この時，Maskには負の無限大がかけられます．これにより，paddingした領域に対しsoftmax後の値を0に近い出力にすることができます．つまり，padding領域のAttention weightを計算しないようにします．最後にValueとの行列乗算をします．

<img src="https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/143078/7dda5594-5bb2-5e2e-566e-3404d722c827.jpeg" width=30%>


#### 2. Multi-Head Attention
Multi-Head Attentionは1つのQuery，Key，Valueを持たせるのではなく，小さいQuery，Key，Valueに分割して，分割した特徴表現を計算します．
構造はシンプルで，Linear層とScaled Dot-Product Attentionを分割した構造を持ちます．
最終的に，分割した出力を1つにまとめてLinear層に渡します．このようにわざわざ分割する理由ですが，モデルが異なる特徴表現の異なる情報についてAttention weightを計算できるためです．


<img src="https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/143078/3c42b52f-aed4-a5af-955b-1156fc27213b.jpeg" width=30%>

In [ ]:
def attention(q, k, v, d_k, mask=None, dec_mask=False):
    scores = np.matmul(q, k.transpose(0, 1, 3, 2)) /  math.sqrt(d_k)
    if mask is not None:
        if dec_mask:
#             print("dec mask shape:", mask.shape)
            mask = mask.reshape([mask.shape[0], 1, mask.shape[1], mask.shape[2]])
        else:
            mask = np.expand_dims(mask, axis=1)
        
        scores[:, :, :, mask[0,0,0,:]==1] = -1e9

    scores = softmax(scores, dim=-1)

    output = np.matmul(scores, v)
    return output
    

class MultiHeadAttention:
    
    def __init__(self, heads, embedding_dim):
        self.h = heads
        self.embedding_dim = embedding_dim
        self.d_k = embedding_dim // heads
        
        self.q_linear = Linear(embedding_dim, embedding_dim)
        self.v_linear = Linear(embedding_dim, embedding_dim)
        self.k_linear = Linear(embedding_dim, embedding_dim)
        
        self.out = Linear(embedding_dim, embedding_dim)
        
    def __call__(self, q, k, v, mask=None, dec_mask=False):
        bs = q.shape[0]
        
        k = self.k_linear(k).reshape([bs, -1, self.h, self.d_k])
        q = self.q_linear(q).reshape([bs, -1, self.h, self.d_k])
        v = self.v_linear(v).reshape([bs, -1, self.h, self.d_k])
        
        k = k.transpose(0, 2, 1, 3)
        q = q.transpose(0, 2, 1, 3)
        v = v.transpose(0, 2, 1, 3)
        
        scores = attention(q, k, v, self.d_k, mask, dec_mask)
        
        concat = scores.transpose(0, 2, 1, 3).reshape([bs, -1, self.embedding_dim])
        output = self.out(concat)
        
        return output

### FeedForwardとLayer Norm

Transformer Blockに存在するその他のモジュールを定義します．

一つ目はFeedForward モジュールです．
これは，通常の全結合層2層から構成されるネットワークのことを表しています．


二つ目はLayer Normalizationです．
Layer Normalizationについては，以前の演習で行っていますので，詳細はそちらを確認してください．

In [ ]:
class FeedForward:
    def __init__(self, embedding_dim, d_ff=2048):
        self.linear_1 = Linear(embedding_dim, d_ff)
        self.linear_2 = Linear(d_ff, embedding_dim)
    
    def __call__(self, x):
        h = relu(self.linear_1(x))
        h = self.linear_2(h)
        return h
    
class Norm:
    def __init__(self, embedding_dim, eps=1e-6):
        self.size = embedding_dim
        self.alpha = np.ones(self.size)
        self.bias = np.zeros(self.size)
        
        self.eps = eps
        
    def __call__(self, x):
        norm = self.alpha * (x - x.mean(axis=-1, keepdims=True)) \
        / (x.std(axis=-1, keepdims=True) + self.eps) + self.bias
        return norm

### Encoder・Decoderの定義

上記で定義したモジュールを用いて，TransformerのEncoderとDecoderを定義します．

In [ ]:
class EncoderLayer:
    def __init__(self, embedding_dim, heads):
        self.norm_1 = Norm(embedding_dim)
        self.norm_2 = Norm(embedding_dim)
        self.attn = MultiHeadAttention(heads, embedding_dim)
        self.ff = FeedForward(embedding_dim)

    def __call__(self, x, mask):
        x2 = self.norm_1(x)
        x = x + self.attn(x2,x2,x2,mask, dec_mask=False)
        x2 = self.norm_2(x)
        x = x + self.ff(x2)
        return x
        
class DecoderLayer:
    def __init__(self, embedding_dim, heads):
        self.norm_1 = Norm(embedding_dim)
        self.norm_2 = Norm(embedding_dim)
        self.norm_3 = Norm(embedding_dim)
        
        self.attn_1 = MultiHeadAttention(heads, embedding_dim)
        self.attn_2 = MultiHeadAttention(heads, embedding_dim)
        self.ff = FeedForward(embedding_dim)

    def __call__(self, x, e_outputs, src_mask, trg_mask):
        x2 = self.norm_1(x)
        x = x + self.attn_1(x2, x2, x2, trg_mask, dec_mask=True)
        x2 = self.norm_2(x)
        x = x + self.attn_2(x2, e_outputs, e_outputs, src_mask, dec_mask=False)
        x2 = self.norm_3(x)
        x = x + self.ff(x2)
        return x

### Transformer

最後にTransformer全体のネットワークを定義します．

In [ ]:
def get_clones(module, N):
    return [copy.deepcopy(module) for i in range(N)]

class Encoder:
    def __init__(self, vocab_size, embedding_dim, N, heads):
        self.N = N
        self.embed = Embedding(vocab_size, embedding_dim)
        self.pe = PositionalEncoder(embedding_dim)
        self.layers = get_clones(EncoderLayer(embedding_dim, heads), N)
        self.norm = Norm(embedding_dim)
        
    def __call__(self, src, mask):
        x = self.embed(src)
        x = self.pe(x)
        for i in range(self.N):
            x = self.layers[i](x, mask)
        return self.norm(x)
    
class Decoder:
    def __init__(self, vocab_size, embedding_dim, N, heads):
        self.N = N
        self.embed = Embedding(vocab_size, embedding_dim)
        self.pe = PositionalEncoder(embedding_dim)
        self.layers = get_clones(DecoderLayer(embedding_dim, heads), N)
        self.norm = Norm(embedding_dim)
        
    def __call__(self, trg, e_outputs, src_mask, trg_mask):
        x = self.embed(trg)
        x = self.pe(x)
        for i in range(self.N):
            x = self.layers[i](x, e_outputs, src_mask, trg_mask)
        return self.norm(x)

class Transformer:
    def __init__(self, vocab_size, embedding_dim, N, heads):
        self.encoder = Encoder(vocab_size, embedding_dim, N, heads)
        self.decoder = Decoder(vocab_size, embedding_dim, N, heads)
        self.out = Linear(embedding_dim, vocab_size)

    def __call__(self, src, trg, src_mask, trg_mask):
        e_outputs = self.encoder(src, src_mask)
        d_output = self.decoder(trg, e_outputs, src_mask, trg_mask)
        output = self.out(d_output)
        return output

def get_model(embedding_dim, heads, n_layers, vocab_size):
    assert embedding_dim % heads == 0
    model = Transformer(vocab_size, embedding_dim, n_layers, heads)
    return model

## Transformerの実行

作成したTransformerモデルと計算機のデータセットを用いて，推論処理を行います．

In [ ]:
# Transformerモデルの準備
embedding_dim = 512
n_layers = 6
heads = 8
vocab_size = len(word2id)

model = get_model(embedding_dim, heads, n_layers, vocab_size)

# データセットの準備
batch_size = 1
test_data = CalcDataset(data_num=10)


accuracy = 0
        
# 評価の実行
for i in range(len(test_data)):
    
    src, trg = test_data[i]
    src = np.expand_dims(src, axis=0)
    trg = np.expand_dims(trg, axis=0)

    trg_input = trg[:, :-1].copy()
    src_mask, trg_mask = create_masks(batch_size, src, trg_input)

    # encoder
    e_output = model.encoder(src, src_mask)

    # decoder
    right = []
    for s in range(4):
        outputs = trg_input[:, :s+1]
        trg_mask_ = trg_mask[:, :s+1, :s+1]
        out = model.out(model.decoder(outputs, e_output, src_mask, trg_mask_))
        out = softmax(out, dim=2)

        if s == 0:
            index = np.argmax(out).item()
        else:
            index = np.argmax(out, axis=2)[0, -1]
        token = id2word[index]

        if token == "<eos>":
            break
        right.append(token)

        trg_input[:, s+1] = float(word2id[token])
    right = "".join(right)

    x = list(src[0])
    try:
        padded_idx_x = x.index(word2id["<pad>"])
    except ValueError:
        padded_idx_x = len(x)
        
    left = "".join(map(lambda c: str(id2word[c]), x[:padded_idx_x]))
    flag = ["F", "T"][eval(left) == int(right)]
    print("{:>7s} = {:>4s} :{}".format(left, right, flag))
    if flag == "T":
        accuracy += 1

print("Accuracy: {:.2f}".format(accuracy / len(test_data)))